<a href="https://colab.research.google.com/github/tatianabazouni/livedrop-tatiana/blob/main/docs/notebooks/llm_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Install dependencies
!pip -q install torch --index-url https://download.pytorch.org/whl/cu121
!pip -q install transformers accelerate sentence-transformers faiss-cpu flask pyngrok bitsandbytes pyyaml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 34.5 MB/s eta 0:00:00


In [2]:
# Cell 2: Import libraries and setup
import os, json,requests, time, threading
from typing import List, Dict
from flask import Flask, request, jsonify
from pyngrok import ngrok
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Globals
PORT = 5000
TOP_K = 4  # top-k retrieval
EMB_MODEL_NAME = "all-MiniLM-L6-v2"  # small, fast embedding model


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


In [3]:
# Cell 3: Knowledge base data as python dictionaries

KNOWLEDGE_BASE = [
    {
        "title": "Document 1: User Registration and Account Management",
        "content": "To create a Shoplite account, users visit the registration page and provide their email address, password, and basic profile information. Email verification is required within 24 hours of registration. Users can choose between buyer accounts (free) or seller accounts (which require additional business verification). The buyer registration is straightforward: username, email, password, and optional phone number for SMS notifications. Password rules enforce a minimum of 8 characters including one uppercase letter and one number. For account recovery, Shoplite supports email-based reset links and optional two-factor authentication (2FA) via TOTP apps (e.g., Google Authenticator). Seller registration begins with company name, tax ID, business address, bank account information for payouts, and uploading identification documents. Shoplite performs automated checks and then a manual review. Once approved, sellers receive an onboarding email with links to seller dashboard, inventory upload templates (CSV), and API keys for integration. Users may upgrade or downgrade account types in Account Settings; downgrades may affect seller features and payout schedules. All accounts must agree to the Terms of Service and Privacy Policy during sign-up. The document includes examples of common registration errors (invalid email formats, password mismatch, duplicate email) and recommended UI messages to reduce user friction. Finally, we describe how Shoplite notifies users about unverified accounts and the process to resend verification links.",
        "id": "doc1"
    },
    {
        "title": "Document 2: Product Search and Filtering Features",
        "content": "Shoplite's search engine combines keyword search, faceted filters, and relevance ranking. Core features include fuzzy matching, stemming, synonyms, and typo tolerance. Filters include category, price range, seller rating, shipping time, availability (in-stock), condition (new/refurbished), and promotions. The UI supports multi-select filters and dynamic counts. Search relevancy uses a weighted mix of title matches, description matches, popularity, conversion rate, and seller reputation. For personalized search, Shoplite applies user history (recently viewed, purchased categories) to boost results. Sorting options include 'Relevance', 'Price: Low to High', 'Price: High to Low', 'Best Sellers', and 'Newest'. Autocomplete leverages prefix matching and trending queries; it returns up to 6 suggestions and throttles suggestions per session to reduce API load. For API consumers, search endpoints accept `q`, `filters` (JSON), `sort`, `page`, and `per_page`. Query performance is optimized with a cached product index updated incrementally as inventory changes. The document also details how to handle out-of-stock SKUs (show 'notify me' and hide from default listings when hidden by seller) and how promotional badges (e.g., 'Free shipping', 'Top Seller') are calculated and presented.",
        "id": "doc2"
    },
    {
        "title": "Document 3: Shopping Cart and Checkout Process",
        "content": "Shoplite’s cart persists across devices for logged-in users; guests maintain a session cart via cookies for 30 days. A cart may contain items from multiple sellers; Shoplite splits the checkout into a unified payment while creating distinct seller-level orders for fulfillment. During checkout, taxes and shipping are estimated based on shipping address and seller settings; final amounts are calculated at order placement. The checkout flow includes address selection, shipping option selection (standard, expedited), payment method, promo code entry, and order review. Payment authorization occurs immediately; capture occurs at the time of order placement except where preauthorization rules apply (e.g., high-risk orders). The system supports saved cards (tokenized) and wallets. For multi-seller orders, Shoplite aggregates totals but tracks each seller’s payout and commission separately. The platform supports invoice generation and sends order confirmation emails to buyer and seller with expected shipping windows. Failed payments trigger retries and email notifications; if payment fails after retries, the order is canceled and inventory restored. The cart UI provides clear error messages when items become unavailable during checkout and supports one-click purchase for stored payment methods.",
        "id": "doc3"
    },
    {
        "title": "Document 4: Payment Methods and Security",
        "content": "Shoplite accepts major card networks and popular digital wallets. Payment processing is handled via partnered payment gateways; card data is never stored on Shoplite servers — tokenization and PCI-compliant partners manage card details. For marketplaces, Shoplite supports split payments via the payment provider, ensuring funds are routed to sellers minus platform commission. Additional fraud mitigation includes AVS check, CVV verification, velocity checks, device fingerprinting, and machine-learning risk signals that trigger manual review for suspect transactions. Refunds are processed through the original payment method and may take 3–10 business days depending on bank processing times. For high-value or flagged transactions, Shoplite may require manual KYC review or additional documents. The Security section covers encryption (TLS 1.2+), secrets management, and recommended developer practices: use short-lived keys, rotate API keys, restrict webhook IPs, and validate payload signatures. The document also explains the dispute workflow: chargeback notifications forwarded to seller with evidence upload portals for response.",
        "id": "doc4"
    },
    {
        "title": "Document 5: Order Tracking and Delivery",
        "content": "After a seller marks an order as shipped, Shoplite integrates with carrier partners to generate tracking numbers and provide real-time status updates. Tracking status includes: Processing, Shipped, In Transit, Out for Delivery, Delivered, and Exception. Shoplite supports auto-tracking by pushing shipping labels via partner APIs or by allowing sellers to upload tracking numbers manually. Delivery estimates are calculated using seller location, buyer location, service level, and lead time. The platform supports fulfillment options: seller fulfillment, platform-fulfilled (Shoplite Logistics), or third-party logistics (3PL) integrations. For multi-seller orders, each item gets its own shipment tracking; the buyer sees them consolidated in the order view. Delivery exceptions (missed delivery, address problem) initiate a notification sequence and request seller or buyer action. For international orders, customs, duties, and VAT handling rules are documented; Shoplite can present DDP (Delivered Duty Paid) or DAP options depending on seller preference. The platform exposes an API for tracking updates and webhooks for status changes to integrated seller systems.",
        "id": "doc5"
    },
    {
        "title": "Document 6: Return and Refund Policies",
        "content": "Shoplite’s standardized return policy for eligible items is a 30-day return window starting from delivery date, unless the seller sets a longer window. Certain categories (perishables, intimate goods) may be non-returnable by default. Return requests require a reason code and may include photo evidence. Once a return is approved, the buyer receives a return authorization and instructions for shipping (prepaid label or buyer-paid depending on seller policy). Refunds are issued after the seller confirms item receipt and inspection; in case of seller dispute, Shoplite’s dispute resolution team intervenes. For seller-initiated refunds (e.g., wrong item), refunds are processed immediately and inventory is adjusted. Partial refunds for damaged items are supported. The document describes the timeline for refunds (typically 3–10 business days), restocking fees (optional for sellers), and cases where the buyer may receive replacement or store credit. The policy also includes guidance for fraudulent return detection and handling inbound returns to third-party warehousing.",
        "id": "doc6"
    },
    {
        "title": "Document 7: Product Reviews and Ratings",
        "content": "Shoplite enables buyers to leave reviews with a star rating (1–5), title, body text, and optional photos. Verified purchases are flagged to help trust. Ratings aggregate at the product and seller level; seller rating is a weighted score combining product ratings, fulfillment performance, and dispute rates. To prevent abuse, Shoplite enforces policies: one review per purchase, detection of suspicious review patterns, and manual moderation for flagged content. Sellers may respond to reviews publicly. Reviews feed into search ranking, with higher-rated products boosted in relevance for certain filters. The platform supports Q&A where prospective buyers ask the seller public questions; sellers and other buyers may answer. Reviews API allows export for sellers to display on external sites. Shoplite uses review sampling to invite buyers to leave reviews via post-delivery emails and push notifications; buyers who report a problem during review submission are directed to open a support ticket instead.",
        "id": "doc7"
    },
    {
        "title": "Document 8: Seller Account Setup and Management",
        "content": "Sellers begin with account creation, provide business details, tax IDs, and bank information for payouts. Shoplite runs automated background checks to validate identity and tax status; the manual review stage covers document veracity. Onboarding includes verifying product categories, acceptable item lists, and compliance checks for restricted goods. The seller dashboard offers step-by-step setup tasks: profile completion, store policies (shipping, returns), inventory upload templates, and pricing rules. Shoplite provides CSV and API options for bulk product listing. Sellers can opt into Shoplite Logistics or use their own shipping integrations. Payout cadence default is weekly, with optional daily payout for high-volume sellers upon credit check approval. Sellers must agree to marketplace policies, fee schedules, and dispute handling procedures. The onboarding guides include best practices for product photography, titles, and descriptions to maximize conversion.",
        "id": "doc8"
    },
    {
        "title": "Document 9: Inventory Management for Sellers",
        "content": "Shoplite supports inventory control with stock-keeping units (SKUs), batch updates, and threshold alerts. Sellers can set `available_stock` per SKU and allow backorder or disable backorder. Inventory updates can be done via the Seller UI, CSV upload, or API webhooks. The system supports multi-warehouse inventory where sellers can allocate stock to different fulfillment centers. When orders place, inventory is reserved (not deducted) until payment capture completes to prevent overselling. Batch imports are validated and produce a preview report with errors. Inventory reporting includes low-stock alerts, turnover reports, and aging inventory suggestions. For integrated sellers, Shoplite offers real-time sync via webhooks to avoid race conditions across channels. Sellers may configure automatic restock notifications to subscribers.",
        "id": "doc9"
    },
    {
        "title": "Document 10: Commission and Fee Structure",
        "content": "Shoplite charges a combination of listing fees (category-dependent), transaction commissions (percentage of sale), and payment processing fees. Commission tiers exist by product category and seller subscription level (Basic, Pro, Enterprise). Example: Basic sellers pay 10% commission and Pro sellers 7% with a monthly subscription fee. Additional fees may apply for promoted listings, fulfillment services, or returns handling. Payout calculations show gross sales, commission deduction, refunds, and net payable amount. Fees are computed at order settlement and visible in seller analytics. The document outlines invoicing cadence, fee disputes, and how promotional rebates are reflected in payouts. Sellers receive a detailed payout statement per payout cycle including order IDs, fees, and adjustments.",
        "id": "doc10"
    },
    {
        "title": "Document 11: Customer Support Procedures",
        "content": "Shoplite’s customer support includes help center articles, in-app chat with bots (first-line), and human agents for escalations. Support operates 24/7 for critical incidents and business hours for standard queries. The support flow encourages self-service: searchable FAQs, guided troubleshooting, and automated flows for common tasks (track order, request return, cancel order). For unresolved issues, tickets are created with SLA rules: 24-hour response for general tickets, 48–72 hours for investigations, and 3–5 business days for appeals. Support agents have role-based dashboards with access to order history, communication logs, and seller responses. The document details escalation matrices (fraud, legal, data breach), templates for common replies, and quality metrics (first response time, resolution time). Agents must follow data privacy rules when requesting documents and only request necessary data.",
        "id": "doc11"
    },
    {
        "title": "Document 12: Mobile App Features",
        "content": "Shoplite’s mobile app provides core marketplace features optimized for small screens: personalized home feed, push notifications for order events, in-app barcode scanner for sellers to add inventory, and one-tap checkout for saved cards. The mobile app supports offline draft carts, camera-based product photography with auto-cropping, and quick-scan QR codes for seller storefront sharing. Push notifications cover order status, promotions, and support messages. The app includes a simplified seller dashboard with order management and quick replies for buyer queries. Some heavy analytics sections are desktop-only for clarity. The document lists permissions the app uses (camera, storage, notifications) and privacy choices presented to users, plus guidance for mobile-first UI/UX patterns used in Shoplite.",
        "id": "doc12"
    },
    {
        "title": "Document 13: API Documentation for Developers",
        "content": "Shoplite provides REST APIs for product listing, inventory updates, order retrieval, and webhooks. API authentication uses API keys created in the seller dashboard; keys may be scoped (read-only, orders-write). The API supports pagination, idempotency tokens for safe retries, and webhooks for order updates and returns. Rate limits vary by plan; Basic plan gets 60 RPM and Pro plan gets higher quotas. The document includes code samples (curl, Python requests) for common tasks: listing a product, retrieving orders, updating inventory, and responding to a webhook challenge. Webhook security includes signing payloads with a secret and replay protection via nonce and timestamp. Error codes are standardized (4xx for client errors, 5xx for server errors) with specific messages for common failure modes.",
        "id": "doc13"
    },
    {
        "title": "Document 14: Security and Privacy Policies",
        "content": "Shoplite’s security model includes encryption-in-transit (TLS 1.2+), encryption-at-rest for sensitive fields, role-based access control, regular vulnerability scanning, and an incident response plan. Personal data processing practices follow privacy-by-design principles; Shoplite supports data subject requests (access, portability, deletion) in line with applicable laws. The document covers password storage (salted hashing), session expiration policies, and secure handling of PII. For developers, there are guidelines on storing secrets, using environment variables, and restricting access. The incident response section outlines notification timelines to affected users and regulators, evidence collection, and remediation steps.",
        "id": "doc14"
    },
    {
        "title": "Document 15: Promotional Codes and Discounts",
        "content": "Shoplite supports promo codes (single use, multi-use), automatic discounts (cart-level), and seller-level promotions. Promo definitions include discount type (percentage, fixed amount), start/end dates, usage limits, and eligible SKUs or categories. Campaigns can be scheduled and A/B tested. The platform supports stacking rules and priority resolution when multiple promotions apply. Analytics show uplift, redemption rate, and incremental sales. For checkout, promo validation runs during cart calculation and returns clear error codes (expired, ineligible, usage limit exceeded). Sellers can allocate a budget to platform promotions and see spend vs. return in analytics.",
        "id": "doc15"
    }
]




In [4]:


# Cell 4: Assisted Prompts Configuration
PROMPTS = {
  "base_retrieval_prompt": {
    "role": "You are a helpful Shoplite customer service assistant.",
    "goal": "Provide accurate, concise answers using only the provided Shoplite documentation snippets.",
    "context_guidelines": [
      "Avoid assumptions",
      "Use only information from the provided document snippets",
      "Cite the document title you used",
      "Keep answers concise (3-5 sentences)",
      "If no relevant context is found, politely refuse and offer next steps"
    ],
    "response_format": "Answer: [Your response based on one document]\nSources: [List document titles referenced]"
  },

  "chitchat_assistant": {
    "role": "You are a friendly Shoplite assistant.",
    "goal": "Respond to greetings and social interactions politely while staying on-brand.",
    "context_guidelines": [
      "Keep responses brief and friendly.",
      "Acknowledge greetings and pleasantries",
      "Avoid technical or policy details unless directly relevant",
      "For questions outside your scope, politely decline and suggest contacting support"
    ],
    "response_format": "Answer: [Friendly response]\nSources: none"
  },

  "multi_doc_synthesis": {
    "role": "You are a Shoplite systems specialist assistant.",
    "goal": "Synthesize details from multiple documents and produce a single authoritative answer with citations.",
    "context_guidelines": [
      "List all documents used in the synthesis",
      "Focus on the most important points across all retrieved documents",
      "Do not list every detail; highlight key facts",
      "Maintain neutrality and avoid bias",
      "Avoid redundancy",
      "Ensure clarity and conciseness",
      "When details conflict, state the conflict and prefer platform-level policies over seller customizations",
      "Include required keywords if present in authoritative answers"
    ],
    "response_format": "Answer: [Synthesis from 2 to 4 documents]\nDetails: [Bulleted facts with document citations]\nSources: [List document titles referenced]"
  },

  "clarify_or_request_more_info": {
    "role": "You are a helpful clarifying assistant for ambiguous queries.",
    "goal": "Ask concise questions to obtain missing context before attempting retrieval.",
    "context_guidelines": [
      "If the user query lacks required info (order ID, product SKU, timeframe), ask for it",
      "Limit clarifying questions to 1–2 so the user can respond quickly"
    ],
    "response_format": "Clarifying question: [Single question]\nExample required info: [order ID, SKU, delivery date, etc.]"
  },

  "refusal_and_escalation": {
    "role": "You are a safety-aware assistant that refuses inappropriate requests and escalates policy issues.",
    "goal": "Refuse to provide or invent information not found in the documentation. Escalate to human support when necessary.",
    "context_guidelines": [
      "If no relevant documents exist, refuse and provide the user with an escalation link or steps",
      "For legal/payment disputes, direct user to open a support ticket"
    ],
    "response_format": "Refusal: [Short explanation]\nNext steps: [Escalation instructions or support link]"
  }
}


In [ ]:
# Cell 5: LLM loading and setup
print("LLM loading...")

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

# Configure 4-bit quantization to fit in Colab GPU memory
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Install bitsandbytes separately to ensure availability
!pip -q install -U bitsandbytes

# Using Llama-2-70b-chat - a larger model
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
print(f"Loading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)



LLM loading...
Loading meta-llama/Llama-2-70b-chat-hf...


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_debugpy_repr.py", line 5

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Install bitsandbytes separately to ensure availability
!pip -q install bitsandbytes

In [ ]:
# Cell 6: RAG pipeline implementation
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Dict, Any

embedding_model = None
faiss_index = None
doc_id_map = {}

def build_rag_pipeline():
    """Initializes the embedding model and builds the FAISS index."""
    global embedding_model, faiss_index, doc_id_map
    print("Building RAG pipeline...")

    # 1. Load Embedding Model
    print("Loading embedding model...")
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

    # 2. Create Embeddings
    doc_contents = [doc['content'] for doc in KNOWLEDGE_BASE]
    embeddings = embedding_model.encode(doc_contents, convert_to_tensor=True)

    # 3. Build FAISS Index
    embedding_dim = embeddings.shape[1]
    faiss_index = faiss.IndexFlatL2(embedding_dim)
    faiss_index.add(embeddings.cpu().numpy())

    # 4. Create a mapping from index ID to document
    for i, doc in enumerate(KNOWLEDGE_BASE):
        doc_id_map[i] = doc

    print(f"✓ FAISS index ready: {faiss_index.ntotal} documents indexed")
    print("✓ RAG pipeline ready!")

def retrieve_docs(query: str, k: int = 4) -> List[Dict[str, Any]]:
    """
    Retrieves the top-k most relevant documents for a given query,
    but returns an empty list if no document meets the relevance threshold.
    """
    query_embedding = embedding_model.encode([query], convert_to_tensor=True)
    distances, indices = faiss_index.search(query_embedding.cpu().numpy(), k)

    # If no documents are found at all, return empty list
    if len(indices[0]) == 0:
        return []

    # This is the key change: Check the distance of the BEST match.
    # If it's too high (meaning not very similar), consider it irrelevant
    # and return nothing. This prevents feeding bad context to the LLM.
    RELEVANCE_THRESHOLD = 1.0  # L2 distance threshold; lower is stricter.

    if distances[0][0] > RELEVANCE_THRESHOLD:
        print(f"⚠️ Top document distance ({distances[0][0]:.2f}) exceeds relevance threshold ({RELEVANCE_THRESHOLD}). Returning no documents.")
        return []

    # Retrieve the actual documents based on indices
    retrieved_docs = [doc_id_map[i] for i in indices[0] if i != -1]

    # Filter out documents that don't meet the threshold (optional, but good for robustness)
    # This step is technically redundant if the top doc is below threshold,
    # but handles cases where subsequent docs might be far off.
    filtered_docs = [retrieved_docs[i] for i in range(len(retrieved_docs)) if distances[0][i] <= RELEVANCE_THRESHOLD]

    return filtered_docs

def generate_response(query: str, context: List[Dict[str, Any]], llm_pipeline) -> str:
    """Generates a response using the LLM based on the query and retrieved context."""
    if not context:
        # Use the refusal prompt if no relevant context is found
        prompt_config = PROMPTS.get("refusal_and_escalation")
        system_message = prompt_config.get("role", "")
        user_message = (
            f"{prompt_config.get('goal', '')}\n\n"
            f"Context Guidelines: {', '.join(prompt_config.get('context_guidelines', []))}\n\n"
            f"User Query: {query}\n\n"
            f"{prompt_config.get('response_format', '')}\nAnswer:"
        )
        full_prompt = f"<s>[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{user_message} [/INST]" # Llama-2 specific format
    else:
        # Determine the prompt configuration based on the number of documents
        if len(context) == 1:
            prompt_config = PROMPTS.get("base_retrieval_prompt")
            system_message = prompt_config.get("role", "")
            context_text = context[0]['content']
            user_message = (
                f"{system_message}\n\n"
                f"Goal: {prompt_config.get('goal', '')}\n\n"
                f"Context Guidelines: {', '.join(prompt_config.get('context_guidelines', []))}\n\n"
                f"Document: {context[0]['title']}\n\n"
                f"Document Content: {context_text}\n\n"
                f"User Query: {query}\n\n"
                f"{prompt_config.get('response_format', '')}\nAnswer:"
            )
             full_prompt = f"<s>[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{user_message} [/INST]" # Llama-2 specific format
        else: # Use multi-doc synthesis for more than one document
            prompt_config = PROMPTS.get("multi_doc_synthesis")
            system_message = prompt_config.get("role", "")
            context_text = "\n\n".join([f"Document: {doc['title']}\nContent: {doc['content']}" for doc in context])
            user_message = (
                f"{system_message}\n\n"
                f"Goal: {prompt_config.get('goal', '')}\n\n"
                f"Context Guidelines: {', '.join(prompt_config.get('context_guidelines', []))}\n\n"
                f"Context Documents:\n{context_text}\n\n"
                f"User Query: {query}\n\n"
                f"{prompt_config.get('response_format', '')}\nAnswer:"
            )
            full_prompt = f"<s>[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{user_message} [/INST]" # Llama-2 specific format


    # Construct the full prompt for the LLM
    # The exact format depends on the LLM's expected input format
    # For Llama-2-70b-chat, a common format is using [INST] [/INST] tags
    # If using a different model, you may need to adjust the prompt template.
    # Refer to your model's documentation for the recommended prompting style.


    # Generate response using the LLM pipeline
    # Adjust max_new_tokens and other parameters as needed
    response = llm_pipeline(
        full_prompt,
        max_new_tokens=256, # Limit the length of the generated response
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

    # Extract the generated text from the response
    # The exact extraction depends on the pipeline's output format
    generated_text = response[0]['generated_text']

    # Post-process the generated text to remove the prompt and extract the answer
    # This is a simple approach; more robust parsing might be needed
    answer_prefix = "Answer:"
    # Find the end of the assistant's response based on the prompt structure
    # For Llama-2, the assistant's response follows the closing [/INST] tag
    prompt_end_marker = "[/INST]"
    if prompt_end_marker in generated_text:
        generated_text = generated_text.split(prompt_end_marker, 1)[1].strip()

    # Remove any leading "Answer:" if the model included it
    if generated_text.startswith(answer_prefix):
        generated_text = generated_text[len(answer_prefix):].strip()


    # Add sources if context was used
    if context:
        sources = ", ".join([doc['title'] for doc in context])
        generated_text += f"\nSources: {sources}"
    else:
        generated_text += "\nSources: none" # Explicitly state no sources if refusal was used

    return generated_text

In [ ]:
# Cell 7: Flask API setup
@app.route('/chat'), @app.route('/ping'), @app.route('/health')

In [ ]:
# Cell 8: ngrok token input and tunnel creation
ngrok_token = input("Enter your ngrok token: ")
# Setup ngrok tunnel

In [ ]:
# Cell 9: Testing and validation
# Test your API endpoints


# Task
Refactor the RAG pipeline code in the notebook into a Python class for better modularity.

## Define a rag class

### Subtask:
Create a Python class, e.g., `RagPipeline`, to encapsulate the RAG functionality.


**Reasoning**:
Define the `RagPipeline` class with a docstring as required by the subtask instructions.



In [ ]:
# Cell 10: Refactor RAG pipeline into a class
class RagPipeline:
    """Encapsulates the RAG pipeline functionality."""
    pass
